In [1]:
import os
import sys
import pandas as pd
import torch

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_classes.architecture import NeuralNetworkArchitecture
from data_classes.enums import OptimizationMethod
from data_classes.experiment import Experiment, SamplingMethod
from data_classes.scenario import Scenario, ScenarioSettings
from data_classes.training_data import InputData
from data_classes.training_config import AdamTrainingConfig, TrainingSetup
from models import SequentialNeuralNetwork

In [2]:
experiment = Experiment(
    SAMPLING_METHOD=SamplingMethod.QMC,
    SCENARIO=Scenario.SUM_SINES
)

scenario_settings = ScenarioSettings(experiment.SCENARIO)

input_data = InputData(scenario_settings.DATA_PATH)

In [3]:
widths = [6,12,24]
depths = [4,8,16]
learning_rates = [0.01,0.001]
lambdas = [1.0e-04,1.0e-05,1.0e-06,1e-07]
training_set_sizes = scenario_settings.TRINING_SET_SIZES

In [4]:
training_settings = []

for width in widths:
    for depth in depths:
        for learning_rate in learning_rates:
            for lambda_ in lambdas:
                for training_set_size in training_set_sizes:
                    nn_arch = NeuralNetworkArchitecture(
                        INPUT_DIM=scenario_settings.INPUT_DIM,
                        OUTPUT_DIM=scenario_settings.OUTPUT_DIM,
                        NUM_HIDDEN_LAYERS=width,
                        DEPTH=depth,
                        ACTIVATION_FUNCTION=torch.nn.Sigmoid
                    )
                    training_config = AdamTrainingConfig(
                        OPTIMIZER=OptimizationMethod.ADAM,
                        LEARNING_RATE=learning_rate,
                        REG_PARAM=lambda_,
                        NUM_EPOCHS=1000,
                        # WEIGHT_DECAY=0.0
                    )

                    training_settings.append(TrainingSetup(
                        nn_architecture=nn_arch,
                        training_config=training_config,
                        training_set_size=training_set_size
                    ))

In [5]:
ts = training_settings[0]
nn = SequentialNeuralNetwork(
    net_arch=ts.nn_architecture
)
training_data = input_data.get_training_and_test_data(
    sampling_method=experiment.SAMPLING_METHOD,
    training_set_size=ts.training_set_size
)
nn.train(settings=ts.training_config, data=training_data)

/Users/janikhrubant/Developer/nn-sampling-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/janikhrubant/Developer/nn-sampling-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Users/janikhrubant/Developer/nn-sampling-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([8192])) that is different to the input size (torch.Si

In [6]:
train_error_results = []
test_error_results = []
for ts in training_settings:
    ts = training_settings[0]
    nn = SequentialNeuralNetwork(
        net_arch=ts.nn_architecture
    )
    training_data = input_data.get_training_and_test_data(
        sampling_method=experiment.SAMPLING_METHOD,
        training_set_size=ts.training_set_size
    )
    nn.train(settings=ts.training_config, data=training_data)

    train_error_results.append(nn.train_error)
    test_error_results.append(nn.test_error)


KeyboardInterrupt: 